#### Вопрос 2

Определить 200 самых частотных слов в тексте (вывести на экран эти слова и их частоты). Среди найденных слов определить слова, не входящие в стандарнтых список стоп-слов.<br>
Вывести эти слова на экран

In [1]:
#Импорт либ
import re
from nltk import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
from nltk import flatten
from typing import List, Tuple

In [2]:
stopwords_set = set(stopwords.words('russian'))
custom_stop_set = {"«","»"}
#Добавляем во множество кастомные кавычки
stopwords_set = stopwords_set | custom_stop_set

In [3]:
def most_common(data : list, count: int) -> List[Tuple[str, int]]:
    """Самые встречающиеся слова"""
    buffer = flatten(data)
    buffer = list(filter(lambda item: item not in stopwords_set, buffer))
    counter = Counter(buffer)
    return counter.most_common(count)

In [4]:
#Чтение файла
with open("./ТОБД21_датасеты/датасеты_к_заданию_2/AnnaKarenina_.txt", "r", encoding="windows-1251") as f:
    data = f.readlines()

In [5]:
words_list = []
#Регулярка, чтоб выкинуть лишние символы
pattern = re.compile(r"[^a-zA-ZA-ЯЁа-яё ]+")
for item in data:
    buffer = re.sub(pattern, '', item.lower())
    words_list.append(word_tokenize(buffer))

In [6]:
most_common(words_list, 200)

#Тут надо как-то на русском сделать еще лемматизацию, но nltk в это не может, а я не помню какая там библиотека для этого нужна
#Но там 3 шага:
#1) Импорт метода с библиотеки
#2) Выставление русского языка
#3) Метод, где на вход принимается str, на выходе str с лемматизацией

[('это', 2222),
 ('сказал', 1411),
 ('левин', 1135),
 ('сказала', 827),
 ('кити', 661),
 ('очень', 573),
 ('вронский', 509),
 ('анна', 499),
 ('алексей', 429),
 ('степан', 423),
 ('аркадьич', 422),
 ('александрович', 395),
 ('время', 366),
 ('мог', 357),
 ('говорил', 357),
 ('руку', 309),
 ('долли', 302),
 ('которые', 295),
 ('чтото', 291),
 ('лицо', 277),
 ('сказать', 276),
 ('дело', 272),
 ('левина', 272),
 ('который', 263),
 ('своей', 251),
 ('знал', 249),
 ('жизни', 235),
 ('говорить', 234),
 ('знаю', 233),
 ('которое', 231),
 ('пред', 224),
 ('хотел', 219),
 ('сергей', 219),
 ('нужно', 217),
 ('человек', 215),
 ('прежде', 215),
 ('глаза', 214),
 ('могу', 214),
 ('видел', 214),
 ('тебе', 213),
 ('тотчас', 211),
 ('чувствовал', 210),
 ('вронского', 204),
 ('одно', 202),
 ('своего', 199),
 ('могла', 199),
 ('свое', 198),
 ('иванович', 191),
 ('думал', 189),
 ('глядя', 189),
 ('говорила', 183),
 ('вместе', 183),
 ('отвечал', 182),
 ('нему', 180),
 ('улыбаясь', 178),
 ('которая', 178),

#### Вопрос 3

В accounts.*.csv найти id, для которого сумма положительных значений в столбце amount наибольшая. Выполнить задание с использованием dask, распараллелив процесс обработки данных.

In [7]:
import dask.dataframe as dd

In [8]:
accounts = dd.read_csv('./ТОБД21_датасеты/датасеты_к_заданию_3/data/accounts.*.csv',
                       dtype= {'amount': 'float64', 'id': 'int64', 'names': 'object'})
#Проверка на то, правильные ли мы тпы указали в dtype
accounts.head()
accounts.tail()

,id,names,amount
999995,43,Dan,933.0
999996,431,Wendy,4913.0
999997,168,Hannah,323.0
999998,308,Norbert,-14.0
999999,353,Ingrid,366.0


In [9]:
#Оставляем только положителньые значения
accounts_fixed = accounts[accounts["amount"] > 0]
accounts_fixed.tail()

,id,names,amount
999994,131,George,153.0
999995,43,Dan,933.0
999996,431,Wendy,4913.0
999997,168,Hannah,323.0
999999,353,Ingrid,366.0


In [10]:
max_id = accounts_fixed.groupby('id')['amount'].sum().idxmax().compute()
max_id

165

### Проверка на адекватность

In [11]:
check = accounts_fixed.groupby('id').sum().sort_values("amount").compute()
check

,amount
id,
250,7.0
28,13.0
437,35.0
442,62.0
141,271.0
...,...
62,94258507.0
49,114515412.0
113,115465183.0
